In [1]:
import os
import csv
from glob import glob
import cv2
import imutils

In [2]:
def read_csv(file_path):
    parent_folder = file_path.rsplit('/', 1)[0]
    with open(file_path, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        return [(parent_folder + '/' + row[0], 
                 parent_folder + '/' + row[1]) for row in csv.reader(f, delimiter=',') if len(row) > 0]

In [3]:
#read_csv("../datasets/nyu2/nyu2_test.csv")

In [4]:
def write_csv(file_path, file_selector="/*/*.jpg", depth_extension=(".jpg", ".png")):
    data_folder = file_path.rsplit('.', 1)[0]
    data_parent_folder = file_path.rsplit('/', 1)[0].replace('/', os.sep)

    def generate_row(image_path):
        image_path = image_path[len(data_parent_folder) + 1:].replace(os.sep, '/')
        return [image_path, image_path[:-len(depth_extension[0])] + depth_extension[1]]

    images = glob((data_folder + file_selector).replace('/', os.sep))
    images.sort(key=lambda e: e.split(os.sep)[-1].split('.')[0])

    with open(file_path, 'w', newline='') as f:
        csv.writer(f, delimiter=',').writerows(
            [generate_row(image) for image in images])

In [5]:
#write_csv("../datasets/nyu2/nyu2_train.csv", "/*/*.jpg", (".jpg", ".png"))
#write_csv("../datasets/nyu2/nyu2_test.csv", "/*_colors.png", ("_colors.png", "_depth.png"))

In [6]:
def preprocess_image(path, w=128, h=128, depth=False, horizontal_flip=False):
    image = cv2.imread(path)
    if depth:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = imutils.resize(image, height=h)
    image_w = image.shape[1]
    image = image[:, (image_w - w) // 2 : (w + image_w) // 2].astype("float")

    if depth:
        image = np.reshape(image, (image.shape[0], image.shape[1], 1))
    
    return (image - image.min()) / (image.max() - image.min())